In [16]:
import tkinter as tk
from tkinter import ttk
import sqlite3

# Function to create the necessary tables
def create_tables():
    conn = sqlite3.connect('billing.db')
    c = conn.cursor()

    # Drop the tables if they exist (for debugging or resetting)
    c.execute("DROP TABLE IF EXISTS customers")
    c.execute("DROP TABLE IF EXISTS products")
    c.execute("DROP TABLE IF EXISTS transactions")

    # Create customers table
    c.execute('''CREATE TABLE customers
                 (CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
                  CustomerName TEXT NOT NULL,
                  Phone TEXT,
                  Email TEXT)''')

    # Create products table
    c.execute('''CREATE TABLE products
                 (ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
                  ProductName TEXT NOT NULL,
                  Price REAL NOT NULL,
                  Quantity INTEGER NOT NULL)''')

    # Create transactions table
    c.execute('''CREATE TABLE transactions
                 (TransactionID INTEGER PRIMARY KEY AUTOINCREMENT,
                  CustomerID INTEGER,
                  ProductID INTEGER,
                  Quantity INTEGER NOT NULL,
                  TotalAmount REAL NOT NULL,
                  Date TEXT NOT NULL,
                  FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
                  FOREIGN KEY (ProductID) REFERENCES products(ProductID))''')

    conn.commit()
    conn.close()

# Call the function to create the tables
create_tables()

# Function to create a custom success message dialog
def show_success_message(message, total_amount=None):
    # Create a new top-level window
    success_window = tk.Toplevel(root)
    success_window.title("Success")
    success_window.geometry("300x200")
    success_window.configure(bg="#e0f7fa")
    success_window.transient(root)  # Make the dialog window transient to the root window
    success_window.grab_set()  # Make the dialog modal

    # Create and place widgets in the success window
    message_label = tk.Label(success_window, text=message, font=('Helvetica', 12, 'bold'), bg="#e0f7fa")
    message_label.pack(pady=20)

    if total_amount is not None:
        total_label = tk.Label(success_window, text=f"Total Amount: ${total_amount:.2f}", font=('Helvetica', 10), bg="#e0f7fa")
        total_label.pack(pady=10)

    ok_button = tk.Button(success_window, text="OK", font=('Helvetica', 10), bg="#4caf50", fg="white", command=success_window.destroy)
    ok_button.pack(pady=10)

# Function to add product to the database
def add_product():
    name = product_name_entry.get()
    price = float(price_entry.get())
    quantity = int(quantity_entry.get())
    conn = sqlite3.connect('billing.db')
    c = conn.cursor()
    c.execute("INSERT INTO products (ProductName, Price, Quantity) VALUES (?, ?, ?)",
              (name, price, quantity))
    conn.commit()
    conn.close()
    product_name_entry.delete(0, tk.END)
    price_entry.delete(0, tk.END)
    quantity_entry.delete(0, tk.END)
    calculate_total()
    show_success_message("Product added successfully!", total_amount=float(total_amount_label.cget("text").replace('$', '')))
    update_product_menu()

# Function to add customer to the database
def add_customer():
    name = customer_name_entry.get()
    phone = phone_entry.get()
    email = email_entry.get()
    conn = sqlite3.connect('billing.db')
    c = conn.cursor()
    c.execute("INSERT INTO customers (CustomerName, Phone, Email) VALUES (?, ?, ?)",
              (name, phone, email))
    conn.commit()
    conn.close()
    customer_name_entry.delete(0, tk.END)
    phone_entry.delete(0, tk.END)
    email_entry.delete(0, tk.END)
    show_success_message("Customer added successfully!")

# Function to add selected product to the bill
def add_to_bill():
    product = product_dropdown.get()
    quantity = int(billing_quantity_entry.get())
    conn = sqlite3.connect('billing.db')
    c = conn.cursor()
    c.execute("SELECT Price FROM products WHERE ProductName=?", (product,))
    price = c.fetchone()[0]
    total = price * quantity
    bill_tree.insert("", "end", values=(product, quantity, f"${total:.2f}"))
    conn.close()

    # Calculate total after adding the item to the bill
    calculate_total()

    # Show success message with the updated total amount
    show_success_message("Item added to the bill!", total_amount=float(total_amount_label.cget("text").replace('$', '')))

# Function to calculate total amount
def calculate_total():
    total = 0
    for child in bill_tree.get_children():
        total += float(bill_tree.item(child, 'values')[2].replace('$', ''))
    total_amount_label.config(text=f"${total:.2f}")

# Function to generate invoice
def generate_invoice():
    calculate_total()
    show_success_message("Invoice generated successfully!", total_amount=float(total_amount_label.cget("text").replace('$', '')))

# Function to update product menu after adding a new product
def update_product_menu():
    conn = sqlite3.connect('billing.db')
    c = conn.cursor()
    c.execute("SELECT ProductName FROM products")
    products = [row[0] for row in c.fetchall()]
    conn.close()
    product_menu['values'] = products

# GUI setup
root = tk.Tk()
root.title("Billing Software")
root.geometry("600x600")

# Set custom fonts and colors
label_font = ('Helvetica', 12)
entry_font = ('Helvetica', 10)
button_font = ('Helvetica', 10, 'bold')
bg_color = "#f7f7f7"
root.configure(bg=bg_color)

# Product Entry Frame
product_frame = ttk.LabelFrame(root, text="Product Entry", padding=(10, 10))
product_frame.pack(pady=10, fill="x", padx=10)

ttk.Label(product_frame, text="Product Name").grid(row=0, column=0, padx=5, pady=5, sticky='w')
ttk.Label(product_frame, text="Price").grid(row=1, column=0, padx=5, pady=5, sticky='w')
ttk.Label(product_frame, text="Quantity").grid(row=2, column=0, padx=5, pady=5, sticky='w')

product_name_entry = ttk.Entry(product_frame)
product_name_entry.grid(row=0, column=1, padx=5, pady=5)
price_entry = ttk.Entry(product_frame)
price_entry.grid(row=1, column=1, padx=5, pady=5)
quantity_entry = ttk.Entry(product_frame)
quantity_entry.grid(row=2, column=1, padx=5, pady=5)

# Use tk.Button for better control over colors
tk.Button(product_frame, text="Add Product", font=button_font, bg="#FF5733", fg="white", command=add_product).grid(row=3, column=1, pady=10)

# Customer Entry Frame
customer_frame = ttk.LabelFrame(root, text="Customer Entry", padding=(10, 10))
customer_frame.pack(pady=10, fill="x", padx=10)

ttk.Label(customer_frame, text="Customer Name").grid(row=0, column=0, padx=5, pady=5, sticky='w')
ttk.Label(customer_frame, text="Phone no.").grid(row=1, column=0, padx=5, pady=5, sticky='w')
ttk.Label(customer_frame, text="Email id.").grid(row=2, column=0, padx=5, pady=5, sticky='w')

customer_name_entry = ttk.Entry(customer_frame)
customer_name_entry.grid(row=0, column=1, padx=5, pady=5)
phone_entry = ttk.Entry(customer_frame)
phone_entry.grid(row=1, column=1, padx=5, pady=5)
email_entry = ttk.Entry(customer_frame)
email_entry.grid(row=2, column=1, padx=5, pady=5)

tk.Button(customer_frame, text="Add Customer", font=button_font, bg="#33C4FF", fg="white", command=add_customer).grid(row=3, column=1, pady=10)

# Billing Frame
billing_frame = ttk.LabelFrame(root, text="Billing", padding=(10, 10))
billing_frame.pack(pady=10, fill="x", padx=10)

ttk.Label(billing_frame, text="Select Product").grid(row=0, column=0, padx=5, pady=5, sticky='w')

# Fetch products from the database for the dropdown
conn = sqlite3.connect('billing.db')
c = conn.cursor()
c.execute("SELECT ProductName FROM products")
products = [row[0] for row in c.fetchall()]
conn.close()

product_dropdown = tk.StringVar()
product_menu = ttk.Combobox(billing_frame, textvariable=product_dropdown, values=products)
product_menu.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(billing_frame, text="Quantity").grid(row=1, column=0, padx=5, pady=5, sticky='w')
billing_quantity_entry = ttk.Entry(billing_frame)
billing_quantity_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Button(billing_frame, text="Add to Bill", font=button_font, bg="#28A745", fg="white", command=add_to_bill).grid(row=2, column=1, pady=10)

# Bill Treeview (for displaying the bill)
bill_tree = ttk.Treeview(root, columns=("Product", "Quantity", "Total"), show="headings")
bill_tree.heading("Product", text="Product")
bill_tree.heading("Quantity", text="Quantity")
bill_tree.heading("Total", text="Total")
bill_tree.pack(pady=10, padx=10, fill="x")

# Invoice Frame
invoice_frame = ttk.LabelFrame(root, text="Invoice", padding=(10, 10))
invoice_frame.pack(pady=10, fill="x", padx=10)

ttk.Label(invoice_frame, text="Total Amount").grid(row=0, column=0, padx=5, pady=5, sticky='w')
total_amount_label = ttk.Label(invoice_frame, text="$0.00")
total_amount_label.grid(row=0, column=1, padx=5, pady=5, sticky='w')

tk.Button(invoice_frame, text="Generate Invoice", font=button_font, bg="#FFD700", fg="black", command=generate_invoice).grid(row=1, column=1, pady=10)

root.mainloop()
